In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import cv2
from tensorflow.keras.preprocessing.image import load_img
import pandas as pd

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=cv2.imread
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=cv2.imread
)

training_set = train_datagen.flow_from_directory(
    'cell-images-for-detecting-malaria/cell-images/',
    target_size=(64, 64),
    batch_size=64,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    'cell-images-for-detecting-malaria/cell-images/',
    target_size=(64, 64),
    batch_size=64,
    class_mode='binary'
)

Model = Sequential()

Model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu', padding='same'))
Model.add(MaxPooling2D(pool_size=(2, 2)))
Model.add(Dropout(rate=0.2))

Model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
Model.add(MaxPooling2D(pool_size=(2, 2)))
Model.add(Dropout(rate=0.2))

Model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
Model.add(MaxPooling2D(pool_size=(2, 2)))
Model.add(Dropout(rate=0.2))

Model.add(Flatten())
Model.add(Dense(units=256, activation='relu'))
Model.add(Dropout(rate=0.2))
Model.add(Dense(units=128, activation='relu'))
Model.add(Dropout(rate=0.2))
Model.add(Dense(units=1, activation='sigmoid'))

Model.summary()

Model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_accuracy', patience=3)

history = Model.fit(
    training_set,
    batch_size=64,
    verbose=1,
    steps_per_epoch=len(training_set),
    epochs=20,
    validation_data=test_set,
    validation_steps=len(test_set),
    callbacks=[earlystop]
)

s = pd.DataFrame(Model.history.history)
print(s)

print('Test Accuracy : {:.2f}%'.format(Model.evaluate(test_set)[1]*100))

Model.save("Malariagit97_final.h5")

# ... (Plotting code)

Model = load_model("Malariagit97_final.h5")
Model.summary()

test_image = load_img("C:/Users/gokul/Project_Malaria/Kaggle_Cnn/cell-images-for-detecting-malaria/cell-images/Uninfected/C2NThinF_IMG_20150604_114815_cell_235.png", target_size=(64, 64))

def Predictions(test_image):
    test_image = np.array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = Model.predict(test_image)

    if result[0][0] == 0:
        return 'Parasitized'
    else:
        return 'Uninfected'

result = Predictions(test_image)
print(result)
